# Train

Notebook for training the model.

## Load data

In [1]:
from sklearn.preprocessing import MaxAbsScaler
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle

import sys
sys.path.append("../src")

from DataHandler import DataHandler
from ModelHandler import ModelHandler
from PostProcessing import PostProcessing
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
 
configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False, 
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "../Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "../tmp/Scattering/002/", 
    "FOLDER_DATA_PATH": "../tmp/Scattering/002/", 
    "N_EPOCHS_TRAINING": 5000,
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Noise level on db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 
dataHandler = DataHandler(configs)

# Se não tiver os dados no formato necessário já organizados, faz a organização
if not os.path.isfile(folderDataPath + "data.p"):
    print("Sorted data not found, creating new file...")
    x, ydet, yclass, ytype, ygroup = dataHandler.loadData(hand_augmentation=configs["USE_HAND_AUGMENTATION"], SNR=configs["SNRdb"])
    print("Data loaded")

    data_mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    strat_classes = np.max(yclass, axis=1)

    train_index, test_index = next(data_mskf.split(x, strat_classes))

    y_train = {
        "detection": ydet[train_index], 
        "type": ytype[train_index], 
        "classification": yclass[train_index], 
        "group": ygroup[train_index]
    }
    
    y_test = {
        "detection": ydet[test_index], 
        "type": ytype[test_index], 
        "classification": yclass[test_index], 
        "group": ygroup[test_index]
    }
    
    dict_data = {
        "x_train": x[train_index], 
        "x_test": x[test_index], 
        "y_train": y_train, 
        "y_test": y_test
    }

    print("Data sorted")

    try:
        os.mkdir(folderDataPath)
    except:
        pass

    pickle.dump(dict_data, open(folderDataPath + "data.p", "wb"))
    print("Data stored")
else:
    dict_data = pickle.load(open(folderDataPath + "data.p", "rb"))

print(dict_data["x_train"].shape)
print(dict_data["x_test"].shape)

modelHandler = ModelHandler(configs)
postProcessing = PostProcessing(configs)
 
X_all = dict_data["x_train"]
ydet_all = dict_data["y_train"]["detection"]
ytype_all = dict_data["y_train"]["type"]
yclass_all = dict_data["y_train"]["classification"]

Using TensorFlow backend.


Sorted data not found, creating new file...
Loading 1


416it [00:49,  8.37it/s]


(832, 16640, 1)
Loading 2


672it [03:27,  3.24it/s]


(2688, 16640, 1)
Loading 3


528it [03:53,  2.26it/s]


(3168, 16640, 1)
Loading 8


96it [01:05,  1.47it/s]


(1728, 16640, 1)
Data loaded
Data sorted
Data stored
(7575, 16640, 1)
(841, 16640, 1)


## Scattering

Builds the scattering model

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from kymatio.keras import Scattering1D

def buildBaseScattering(input_shape):
    '''
        Source: https://github.com/kymatio/kymatio/blob/master/examples/1d/classif_keras.py
    '''
    log_eps = 1e-6

    input = Input(shape=(input_shape,))
    x = Scattering1D(10, 14)(input) # Changed J from 8 to 10 -> Results in a flatten with 544 parameters (the original with convolutions has 520)
    ###############################################################################
    # Since it does not carry useful information, we remove the zeroth-order
    # scattering coefficients, which are always placed in the first channel of
    # the scattering transform.

    x = Lambda(lambda x: x[..., 1:, :])(x)

    # To increase discriminability, we take the logarithm of the scattering
    # coefficients (after adding a small constant to make sure nothing blows up
    # when scattering coefficients are close to zero). This is known as the
    # log-scattering transform.

    x = Lambda(lambda x: tf.math.log(tf.abs(x) + log_eps))(x)

    ###############################################################################
    # We then average along the last dimension (time) to get a time-shift
    # invariant representation.

    x = GlobalAveragePooling1D(data_format='channels_first')(x)

    model = Model(inputs = input, outputs=x)

    return model

if not os.path.isfile(configs["FOLDER_PATH"] + 'scattering_model.h5'):
    scattering_extract = buildBaseScattering(X_all.shape[1])
    scattering_extract.save(configs["FOLDER_PATH"] + 'scattering_model.h5')
else:
    scattering_extract = modelHandler.loadModel(configs["FOLDER_PATH"] + 'scattering_model.h5')

scattering_extract.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16640)]           0         
_________________________________________________________________
scattering1d (Scattering1D)  (None, 628, 17)           0         
_________________________________________________________________
lambda (Lambda)              (None, 627, 17)           0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 627, 17)           0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 627)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## New Train

In [ ]:
fold = 0
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
strat_classes = np.max(yclass_all, axis=1)
print(strat_classes.shape)

for train_index, validation_index in mskf.split(X_all, strat_classes):
    fold += 1

    print(f"---------- FOLD {fold} -------------")

    scaler = MaxAbsScaler()
    scaler.fit(np.squeeze(X_all[train_index], axis=2))
    x_train = np.expand_dims(scaler.transform(np.squeeze(X_all[train_index], axis=2)), axis=2)
    x_validation = np.expand_dims(scaler.transform(np.squeeze(X_all[validation_index], axis=2)), axis=2)
    
    x_train = scattering_extract.predict(x_train)
    x_validation = scattering_extract.predict(x_validation)

    print(x_train.shape)

    y_train, y_validation = {}, {}
    y_train["detection"] = ydet_all[train_index]
    y_validation["detection"] = ydet_all[validation_index]
    y_train["type"] = ytype_all[train_index]
    y_validation["type"] = ytype_all[validation_index]
    y_train["classification"] = yclass_all[train_index]
    y_validation["classification"] = yclass_all[validation_index]

    yclass_weights = calculating_class_weights(np.max(y_train["classification"], axis=1))

    print(yclass_weights)
    
    folderPath = configs["FOLDER_PATH"] + str(fold) + "/"
    try:
        os.mkdir(folderPath)
    except:
        pass

    np.save(folderPath + "train_index.npy", train_index)
    np.save(folderPath + "validation_index.npy", validation_index)
    
    tensorboard_callback = TensorBoard(log_dir='./' + configs["FOLDER_PATH"] + '/logs')

    if configs["INITIAL_EPOCH"] > 0:
        model = ModelHandler.loadModel(folderPath + 'model_{0}.h5'.format(configs["INITIAL_EPOCH"]))
    else:
        model = modelHandler.buildScatteringOutput(input_shape=x_train.shape[1])
 
    model.summary()
 
    fileEpoch = configs["INITIAL_EPOCH"]
    while fileEpoch < configs["TOTAL_MAX_EPOCHS"]:
        fileEpoch += configs["N_EPOCHS_TRAINING"]      

        if not os.path.isfile(folderPath + 'model.h5'):
            for subtask in ['type', 'detection', 'classification']:
                print(f"FOLD {fold}: Training {subtask}")
                
                freeze(model, task_name=subtask)
                model.compile(optimizer = Adam(), \
                              loss = [ModelHandler.sumSquaredError, "categorical_crossentropy", ModelHandler.get_bce_weighted_loss(yclass_weights)], \
                              metrics=[['mean_squared_error'], ['categorical_accuracy'], ['binary_accuracy']])
                
                early_stopping_callback = EarlyStopping(monitor=f"val_{subtask}_loss", patience=50, verbose=True, restore_best_weights=True)
                hist_opt = model.fit(x=x_train, y=[y_train["detection"], y_train["type"], y_train["classification"]], \
                                    validation_data=(x_validation, [y_validation["detection"], y_validation["type"], y_validation["classification"]]), \
                                    epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[early_stopping_callback, tensorboard_callback], batch_size=32)
            
            model.save(folderPath + 'model.h5')
  
    del model, y_validation, y_train, x_validation, x_train

## Old Train

In [7]:
fold = 0
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
strat_classes = np.max(yclass_all, axis=1)
print(strat_classes.shape)

for train_index, validation_index in mskf.split(X_all, strat_classes):
    fold += 1

    print(f"---------- FOLD {fold} -------------")

    if fold != 1:
        scaler = MaxAbsScaler()
        scaler.fit(np.squeeze(X_all[train_index], axis=2))
        x_train = np.expand_dims(scaler.transform(np.squeeze(X_all[train_index], axis=2)), axis=2)
        x_validation = np.expand_dims(scaler.transform(np.squeeze(X_all[validation_index], axis=2)), axis=2)
        
        x_train = scattering_extract.predict(x_train)
        x_validation = scattering_extract.predict(x_validation)

    y_train, y_validation = {}, {}
    y_train["detection"] = ydet_all[train_index]
    y_validation["detection"] = ydet_all[validation_index]
    y_train["type"] = ytype_all[train_index]
    y_validation["type"] = ytype_all[validation_index]
    y_train["classification"] = yclass_all[train_index]
    y_validation["classification"] = yclass_all[validation_index]

    yclass_weights = calculating_class_weights(np.max(y_train["classification"], axis=1))
    print(yclass_weights)
    
    folderPath = configs["FOLDER_PATH"] + str(fold) + "/"
    try:
        os.mkdir(folderPath)
    except:
        pass

    np.save(folderPath + "train_index.npy", train_index)
    np.save(folderPath + "validation_index.npy", validation_index)
    
    tensorboard_callback = TensorBoard(log_dir='./' + configs["FOLDER_PATH"] + '/logs')
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=50, verbose=True, restore_best_weights=True)
    classification_early_stopping_callback = EarlyStopping(monitor='val_classification_loss', patience=50, verbose=True, restore_best_weights=True)
    detection_early_stopping_callback = EarlyStopping(monitor='val_detection_loss', patience=50, verbose=True, restore_best_weights=True)
    type_early_stopping_callback = EarlyStopping(monitor='val_type_loss', patience=50, verbose=True, restore_best_weights=True)

    if configs["INITIAL_EPOCH"] > 0:
        model = ModelHandler.loadModel(folderPath + 'model_{0}.h5'.format(configs["INITIAL_EPOCH"]))
    else:
        model = modelHandler.buildScatteringOutput(input_shape=x_train.shape[1])
 
    model.summary()
 
    fileEpoch = configs["INITIAL_EPOCH"]
    while fileEpoch < configs["TOTAL_MAX_EPOCHS"]:
        fileEpoch += configs["N_EPOCHS_TRAINING"]

        if not os.path.isfile(folderPath + 'model.h5'):
            model.compile(optimizer = Adam(), \
                          loss = [ModelHandler.sumSquaredError, "categorical_crossentropy", ModelHandler.get_bce_weighted_loss(yclass_weights)], \
                          metrics=[['mean_squared_error'], ['categorical_accuracy'], ['binary_accuracy']])
            hist = model.fit(x=x_train, y=[y_train["detection"], y_train["type"], y_train["classification"]], \
                             validation_data=(x_validation, [y_validation["detection"], y_validation["type"], y_validation["classification"]]), \
                             epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[early_stopping_callback, tensorboard_callback], batch_size=32)

            model.save(folderPath + 'model.h5')
        else: # If this model already exists, loads it
            model = ModelHandler.loadModel(folderPath + "model.h5")        

        if not os.path.isfile(folderPath + 'model_class_opt.h5'):
            print(f"FOLD {fold}: CLASSIFICATION FINE TUNNING PHASE")
            freeze(model)
            model.compile(optimizer = Adam(learning_rate=0.00001), \
                          loss = [ModelHandler.sumSquaredError, "categorical_crossentropy", ModelHandler.get_bce_weighted_loss(yclass_weights)], \
                          metrics=[['mean_squared_error'], ['categorical_accuracy'], ['binary_accuracy']])

            hist_opt = model.fit(x=x_train, y=[y_train["detection"], y_train["type"], y_train["classification"]], \
                                 validation_data=(x_validation, [y_validation["detection"], y_validation["type"], y_validation["classification"]]), \
                                 epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[classification_early_stopping_callback, tensorboard_callback], batch_size=32)
          
            model.save(folderPath + 'model_class_opt.h5')
  
    del model, y_validation, y_train, x_validation, x_train

(18723, 26)
---------- FOLD 1 -------------
[[ 0.54978454  5.52163934]
 [ 0.55071942  5.42907801]
 [ 0.50298668 84.205     ]
 [ 0.56039531  4.63939394]
 [ 0.57568196  3.8032972 ]
 [ 0.50388965 64.77307692]
 [ 0.51866338 13.89521452]
 [ 0.61217739  2.72861309]
 [ 0.54788861  5.72044837]
 [ 0.50543217 46.52209945]
 [ 0.51160459 22.04319372]
 [ 0.55639619  4.93292326]
 [ 0.66946255  1.97525217]
 [ 0.57560325  3.80673599]
 [ 0.50695364 36.45238095]
 [ 0.57678608  3.75579839]
 [ 0.56949141  4.09756691]
 [ 0.51735684 14.90353982]
 [ 0.52661038  9.89482961]
 [ 0.51764308 14.66986063]
 [ 0.52740198  9.62342857]
 [ 0.53065919  8.65416238]
 [ 0.53473678  7.69698355]
 [ 0.52392359 10.94993498]
 [ 0.53688472  7.27787381]
 [ 0.53324679  8.01952381]]
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None

KeyboardInterrupt: 